## Linear classifier benchmarking

In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.linalg import eigh
import hyperdt.benchmarking as benchmarking
from hyperdt.product_space_DT import ProductSpace
from hyperdt.product_space_perceptron import mix_curv_perceptron
from hyperdt.product_space_svm import mix_curv_svm

### Product space perceptron and SVM 

In [47]:
# Generate data
sig = [(5, -1.), (4, 0.), (3, 1.)]
ps = ProductSpace(sig, seed=0)
ps.sample_clusters(1000, 4, cov_scale=0.3)
ps.split_data()

In [48]:
mix_component = benchmarking.sig_to_mix_component(sig)
embed_data = benchmarking.make_embed_data(ps.X, ps.X_train, ps.X_test, ps.y_train, ps.y_test, sig)

In [49]:
# Fit product space perceptron
ps_perc = mix_curv_perceptron(mix_component, embed_data, multiclass=True, max_round=100, max_update=1000)
ps_perc_score = ps_perc.process_data()


 1 rounds finished.

 2 rounds finished.

 3 rounds finished.

 4 rounds finished.

 5 rounds finished.
Platt probability computed

 1 rounds finished.

 2 rounds finished.

 3 rounds finished.

 4 rounds finished.

 5 rounds finished.
Platt probability computed

 1 rounds finished.

 2 rounds finished.

 3 rounds finished.

 4 rounds finished.

 5 rounds finished.
Platt probability computed

 1 rounds finished.

 2 rounds finished.

 3 rounds finished.

 4 rounds finished.

 5 rounds finished.

 6 rounds finished.

 7 rounds finished.

 8 rounds finished.

 9 rounds finished.
Platt probability computed
F1 score: 0.03625502111442991 total number of testing samples: 200


In [50]:
# Fit product space SVM
ps_svm = mix_curv_svm(mix_component, embed_data)
ps_svm_score = ps_svm.process_data()

Mix curv SVM F1 score: 0.12962962962962962


### Benchmarking for HyperDT/RF, sklearn DT/RF, perceptron and SVM

In [9]:
NUM_POINTS = 1000
NUM_CLASSES = 4
signatures = [
    [(5, -1), (5, -1)],
    [(5, 1), (5, 1)],
    [(5, -1), (5, 1)],
    [(2, -1), (2, -1), (2, -1), (2, -1), (2, -1)],
    [(2, 1), (2, 1), (2, 1), (2, 1), (2, 1)],
    [(2, -1), (2, -1), (2, 0), (2, 1), (2, 1)]
]

In [10]:
seed = 0
results = []

scores_tuple = benchmarking.compute_scores_by_signature(signatures, NUM_POINTS, NUM_CLASSES, seed=seed,
                                                        max_depth=3, n_seeds=10, metric="f1")
rnd_seeds = scores_tuple[0]
psdt_scores_by_signature = scores_tuple[1]
psrf_scores_by_signature = scores_tuple[2]
dt_scores_by_signature = scores_tuple[3]
rf_scores_by_signature = scores_tuple[4]
for signature, psdt_score, psrf_score, dt_score, rf_score in zip(signatures, psdt_scores_by_signature,
                                                                       psrf_scores_by_signature, dt_scores_by_signature,
                                                                       rf_scores_by_signature):
    results.append({
        "signature": signature,
        "psdt_score": psdt_score,
        "psrf_score": psrf_score,
        "dt_score": dt_score,
        "rf_score": rf_score
    })

results = pd.DataFrame(results)
results

  2%|▏         | 1/60 [00:06<06:18,  6.41s/it]

KeyboardInterrupt: 

In [ ]:
for signature, (i, row) in zip(signatures, results.iterrows()):
    psdt_scores = row["psdt_score"]
    psrf_scores = row["psrf_score"]
    dt_scores = row["dt_score"]
    rf_scores = row["rf_score"]
    s = []
    for component in signature:
        if component[1] < 0:
            s.append("H(" + f"{component[0]}, {-component[1]}" + ")")
        elif component[1] == 0:
            s.append("E(" + f"{component[0]}" + ")")
        else:
            s.append("S(" + f"{component[0]}, {component[1]}" + ")")
    print(" x ".join(s))
    print(f"Avg PSDT Score: {np.mean(psdt_scores) * 100:.1f} +/- {np.std(psdt_scores) / np.sqrt(len(psdt_scores)) * 1.96 * 100 :.1f}", end=", ")
    print(f"Avg DT Score: {np.mean(dt_scores) * 100:.1f} +/- {np.std(dt_scores) / np.sqrt(len(dt_scores)) * 1.96 * 100 :.1f}")
    
    t, p = stats.ttest_rel(psdt_scores, dt_scores)
    if p < .05:
        print("PSDT and DT are significantly different")
    
    print(f"Avg PSRF Score: {np.mean(psrf_scores) * 100:.1f} +/- {np.std(psrf_scores) / np.sqrt(len(psrf_scores)) * 1.96 * 100 :.1f}", end=", ")
    print(f"Avg RF Score: {np.mean(rf_scores) * 100:.1f} +/- {np.std(rf_scores) / np.sqrt(len(rf_scores)) * 1.96 * 100 :.1f}")

    t, p = stats.ttest_rel(psrf_scores, rf_scores)
    if p < .05:
        print("PSRF and RF are significantly different")
    print("")